In [1]:
'''
Script to get the metadata for each institution that each JPL author has a direct connection to (via affiliation)
'''

'\nScript to get the metadata for each institution that each JPL author has a direct connection to (via affiliation)\n'

In [2]:
import csv
import requests

from helper_funcs import *
from constants import *

In [3]:
individual_ids = parse_csv_of_individuals(JPL_individual_fp)

In [4]:
#### QUERY OPEN ALEX TO GET THE NECESSARY DATA ####

In [5]:
# Function to retrieve detailed information about an author
def get_institution_info(institution_id):
    url = f"{base_url}/institutions/{institution_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

In [6]:
raw_openalex_data = read_json_from_local(person_raw_openalex_data_fp)

In [7]:
inst_to_res = {}

def process_institution(affiliation_metadata,for_last):    
    if for_last:
        i_id = affiliation_metadata['id'].rsplit("/", 1)[-1]
    else:
        i_id = affiliation_metadata['institution']['id'].rsplit("/", 1)[-1]
    
    if i_id not in inst_to_res:
        metadata = get_institution_info(i_id)

        inst_type = ORG
        
        for associated in metadata['associated_institutions']:
            if associated['relationship'] == "parent": #defaults to group
                inst_type = GROUP
                break
                
        inst_to_res[i_id] = {
                "inst_type":inst_type,
                "metadata":metadata
            }

for person, metadata_obj in raw_openalex_data.items():
    for affiliation in metadata_obj['affiliations']:
        process_institution(affiliation,False)
            
    for last_known_affiliation in metadata_obj['last_known_institutions']:
        process_institution(last_known_affiliation,True)

In [8]:
write_json_to_local(inst_to_res,inst_raw_openalex_data_fp)